In [1]:
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import *
import ast

In [4]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''
    # given query, key,value it finds the rightful weighted component of v to get the attention applied ouput
    #q,v,k- batch X length of sequence X features or encoding
    #attention sholuld be -batchX7X7
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
#         print(k.transpose(1,2).shape)

        attn = torch.bmm(q, k.transpose(1, 2)) 
#         print(attn.shape)
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
#         print(str(attn.shape)+" "+str(v.shape))
        output = torch.bmm(attn, v)

        return output, attn
SDP=ScaledDotProductAttention(5)
Ss=SDP(torch.zeros(5,6,100),torch.zeros(5,6,100),torch.zeros(5,6,100))
# print(Ss[0].shape)
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
#         print(str(sz_b)+"die")

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
#         print("v-"+str(v.shape))
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)
#         print(q.shape,k.shape,v.shape)
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn
MHA=MultiHeadAttention(4,15,15,15)
op=MHA(torch.zeros(5,7,15),torch.zeros(5,7,15),torch.zeros(5,7,15))
# print(op[0].shape)
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1) # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1) # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
#         print("FCC-"+str(output.shape))
#         print("FFC_out-"+str(self.w_1(output).shape))
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output

    

In [5]:
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
#         print("ENC_o")
#         print(enc_output.shape)


        enc_output = self.pos_ffn(enc_output)


        return enc_output, enc_slf_attn
    
XX=EncoderLayer(15,10,4,10,10)

zz=XX(torch.zeros(5,7,15))
# print("ENc")
# print(zz[0].shape)
# print("start")
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(self,n_modality,d_model,n_head,d_k,d_v,dropout,n_layers,d_inner=500):
        #d_model - number of features in input 100 here
        #n_head - number of heads of multihaded attention
        #d_k=d_q=  number of features in query, key
        #d_v = number of features in value whose weighted(attentioned) sum we gonna take
        

        super().__init__()
        self.n_modality=n_modality
#         self.stn=nn.ModuleList([SpatialTransformer(3, (240,240), 8) for _ in range(n_ref)])
        
        self.layer_stack = nn.ModuleList([EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout) 
                                          for _ in range(n_layers)])
        self.em=nn.Linear(48,100)
        #self.em1=nn.Linear(225,100)
        self.fc1=nn.Linear(d_model*n_modality,300)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(300,100)
        self.fc3=nn.Linear(100,3)
#         self.fc4=nn.Linear(50,3)
        self.bn1 = nn.BatchNorm1d(num_features=300)
        self.bn2 = nn.BatchNorm1d(num_features=100)
        self.softmax=nn.Softmax(1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embeddings1,embeddings2 ):


        
        encodings_total=[embeddings1,self.em(embeddings2)]

        enc_output=torch.stack(encodings_total,0)

        
        enc_output=enc_output.permute(1,0,2)
#         print("encoding_OUTPUT2-"+str(enc_output.shape))

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output,non_pad_mask=None,slf_attn_mask=None)
           
        
        final_input=enc_output.reshape(enc_output.shape[0],-1)

        final=self.relu(self.fc3(self.bn2(self.relu((self.fc2(self.bn1(self.relu(self.fc1(final_input)))))))))
        
        return(final)
     

In [6]:
e1=torch.zeros(4,100)
e2=torch.zeros(4,100)
e3=torch.stack([e1,e2],0)
e3.shape

torch.Size([2, 4, 100])

In [7]:
feature_set_MLP=pd.read_csv('BCLL_MLP_features_ankit.csv')
feature_set_3d=pd.read_csv('AnkitBCLL_3d.csv')
feature_set_3d.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47
0,hint1,0.000128,0.003666,0.000086,-0.000142,0.013800,0.000378,-0.000040,3.907078e-03,-0.000168,...,0.000020,-0.000014,-0.008416,0.000020,-0.000204,0.005934,-0.000014,0.000322,0.003235,0.000067
1,ywhae,-0.000080,0.000155,0.003896,0.000124,0.001357,0.003743,0.000142,-5.598767e-04,-0.002663,...,0.003278,-0.000108,-0.001078,0.002478,0.000050,0.000687,0.005183,-0.000012,0.000014,-0.001313
2,kat2b,-0.000067,0.003084,0.003566,-0.000042,0.005316,0.004484,-0.000035,8.720113e-04,-0.003647,...,0.002714,-0.000130,-0.002824,0.003957,0.000125,0.003588,0.005358,0.000133,-0.000062,0.000689
3,cib1,0.000136,0.000098,0.024427,0.000034,-0.000053,0.021380,-0.000461,1.194030e-04,-0.022567,...,0.025997,0.000306,0.000063,0.020553,0.000282,-0.000167,0.023225,-0.000222,-0.000087,-0.007981
4,cyp1a1,-0.000320,0.000653,0.009505,-0.000159,0.000983,0.010270,-0.000024,5.357184e-07,-0.007808,...,0.006218,-0.000164,-0.000459,0.007649,-0.000432,0.000244,0.009911,-0.000121,0.000433,-0.000752


In [8]:
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,125133.351562,357573.156250,0.000000,587084.000000,0.0,315726.906250,0.0,175430.843750,268782.625000,...,442658.781250,0.0,465458.937500,370437.156250,0.0,0.0,472186.937500,0.0,0.0,466427.312500
1,rabggta,20101.933594,62892.046875,0.000000,103993.031250,0.0,55333.929688,0.0,30097.763672,48279.539062,...,79100.062500,0.0,82092.179688,65288.472656,0.0,0.0,83334.328125,0.0,0.0,81504.468750
2,ywhae,6624.807129,14475.721680,0.000000,24332.548828,0.0,13422.669922,0.0,9587.588867,10549.949219,...,18221.716797,0.0,19292.927734,14790.755859,0.0,0.0,19355.056641,0.0,0.0,19177.373047
3,kat2b,4142.221680,8753.067383,0.000000,14703.916016,0.0,8161.906250,0.0,5902.970215,6346.573730,...,11016.152344,0.0,11690.555664,8958.273438,0.0,0.0,11691.131836,0.0,0.0,11626.661133
4,smad5,1322.737061,2288.863037,26.059177,3835.314453,0.0,2206.577148,0.0,1791.832397,1568.654663,...,2822.048828,0.0,3072.452148,2335.262207,0.0,0.0,3047.292969,0.0,0.0,3087.742920


In [9]:
header_of_MLP=['tag_'+str(i) for i in range(feature_set_MLP.shape[1]-1)]
features_MLP=np.array(feature_set_MLP[header_of_MLP])
gene_MLP=feature_set_MLP['gene_name']
print(features_MLP.shape)
print(len(gene_MLP))

(4655, 100)
4655


In [10]:
dictionary_MLP={}
u=0
for gn in gene_MLP:
    dictionary_MLP[gn]=features_MLP[u]
    u=u+1

In [11]:
header_of_3d=['tag_'+str(i) for i in range(feature_set_3d.shape[1]-1)]
features_3d=np.array(feature_set_3d[header_of_3d])
gene_3d=feature_set_3d['gene_name']
print(features_3d.shape)
print(len(gene_3d))


(2244, 48)
2244


In [12]:
dictionary_3d={}
u=f=0
for gn in gene_3d:
    if gn in dictionary_3d.keys():
#         print(gn)
        f=f+1
    dictionary_3d[gn]=features_3d[u]
    u=u+1
print(u)

2244


In [13]:
print(len(dictionary_3d.keys()))

1935


In [13]:
fil=open('../../Multi-modality/Model/BCLL/uni model/MLP/label0.txt','r')
tmp=list()
for line in fil:
	tmp.append(int(line))

label_BCLL=np.array(tmp)
print(label_BCLL.shape)

(4655,)


In [14]:
class Sequenceloader(data.Dataset):
    def __init__(self,GN,Feat,label):
        self.gene_names=GN
        self.features_mlp=Feat
        self.label=label
        self.coincdgene_name=[]
        self.coincidfeature_MLP=[]
        self.coincidfeature_3d=[]
        self.coincidlabel=[]
        for i in range(len(self.gene_names)):
            u=self.gene_names[i]
            if u in dictionary_3d.keys():
                
                if np.array(self.label[i])==0:
                    ch=50
                elif np.array(self.label[i])==1: 
                    ch=7
                else:
                    ch=1
                    
                for jj in range(ch):
                    self.coincdgene_name.append(u)
                    self.coincidfeature_MLP.append(self.features_mlp[i])
                    self.coincidfeature_3d.append(dictionary_3d[u])
                    self.coincidlabel.append(self.label[i])
        
        
    
    def __len__(self):
#         print(len(self.dataset))
        return len(self.coincdgene_name)       

    def __getitem__(self, index):
         return np.array(self.coincidfeature_MLP[index]),np.array(self.coincidfeature_3d[index]),np.array(self.coincidlabel[index])
#         print(self.dataset['gen_name'][index])          
#         try:
           
#         except :
            
total_set=Sequenceloader(gene_MLP,features_MLP,label_BCLL)  
a=b=c=0
for x,y,z in total_set:
    if(z==0):
        a=a+1
    elif z==1:
        b=b+1
    else:
        c=c+1
print(a,b,c)
    

1950 2023 1916


In [15]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(total_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size, 
                                           sampler=train_sampler,drop_last=True)
validation_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size,
                                                sampler=valid_sampler,drop_last=True)


In [16]:
for a,b,c in train_loader:
    print(a.shape,b.shape,c.shape)

torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size

torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size

torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size

torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size

In [17]:
def test(test_loader,model):

    total_imgs=0;
    total_corrects=0
    u=0
    nb_classes=3
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i1,i2,label in test_loader:
                

        output=model(i1.to(device).float(),i2.to(device).float())
        total_imgs=total_imgs+label.shape[0]
        z=torch.max(output,1)[1]==label.to(device)
        _, preds = torch.max(output, 1)
#         print(output.shape)
 
        num_corrects=torch.sum(z)
        total_corrects=total_corrects+num_corrects
        for t, p in zip(label.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1


        u=u+1
    
    
    print(confusion_matrix)
    return(total_corrects,total_imgs)

In [18]:
device='cuda:7'
model_3dXMLP=Encoder(2,100,4,300,300,True,4).to(device)
uz=torch.rand(4,100).to(device)
vz=torch.rand(4,48).to(device)
model_3dXMLP(uz,vz).shape


torch.Size([4, 3])

In [20]:

#model_3dXMLP.load_state_dict(torch.load(Path('1ANKIT_BCLL COMBO_3DXMLP.pt')))
optim_params = [
    {'params': model_3dXMLP.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)
criterion = nn.CrossEntropyLoss()

In [21]:
epoch=10000
for i in range(epoch):
    torch.save(model_3dXMLP.state_dict(), '1ANKIT_BCLL COMBO_3DXMLP.pt')
    print("Accuracy-"+str(test(validation_loader,model_3dXMLP)))
    total_loss=0
    for inp1,inp2,lab in train_loader:

        

        output=model_3dXMLP(inp1.to(device).float(),inp2.to(device).float())

        loss_batch=criterion(output,lab.to(device))/4
        optimizer.zero_grad()  
        loss_batch.backward()
        optimizer.step()
        total_loss+=loss_batch
       
    print(total_loss)
    
    
        
        

tensor([[383.,  19.,  10.],
        [ 10., 321.,  57.],
        [ 19., 108., 249.]])
Accuracy-(tensor(953, device='cuda:7'), 1176)
tensor(98.6729, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,  17.,  13.],
        [  7., 333.,  48.],
        [ 16.,  95., 265.]])
Accuracy-(tensor(980, device='cuda:7'), 1176)
tensor(100.0590, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[400.,   7.,   6.],
        [ 11., 335.,  42.],
        [ 15.,  92., 268.]])
Accuracy-(tensor(1003, device='cuda:7'), 1176)
tensor(100.8282, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[397.,   9.,   7.],
        [ 15., 319.,  54.],
        [ 18.,  83., 274.]])
Accuracy-(tensor(990, device='cuda:7'), 1176)
tensor(101.0141, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[387.,  16.,  10.],
        [ 13., 323.,  51.],
        [ 12., 105., 259.]])
Accuracy-(tensor(969, device='cuda:7'), 1176)
tensor(95.0938, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[388.,  10.,  15.],
        [  7., 323.,  57.],


tensor(97.4628, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[397.,  10.,   6.],
        [ 10., 327.,  50.],
        [ 16.,  77., 283.]])
Accuracy-(tensor(1007, device='cuda:7'), 1176)
tensor(102.4357, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  11.,  17.],
        [ 10., 331.,  46.],
        [ 15.,  88., 273.]])
Accuracy-(tensor(989, device='cuda:7'), 1176)
tensor(98.5208, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[380.,  21.,  12.],
        [  8., 310.,  69.],
        [ 16.,  82., 278.]])
Accuracy-(tensor(968, device='cuda:7'), 1176)
tensor(100.4893, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[391.,  12.,  10.],
        [ 11., 332.,  45.],
        [ 20., 108., 247.]])
Accuracy-(tensor(970, device='cuda:7'), 1176)
tensor(101.0600, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  21.,   7.],
        [  7., 327.,  53.],
        [ 13.,  93., 270.]])
Accuracy-(tensor(982, device='cuda:7'), 1176)
tensor(96.4220, device='cuda:7', grad_fn=<AddBackward0>)

tensor(99.9228, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[389.,  16.,   8.],
        [ 10., 314.,  63.],
        [ 14.,  81., 281.]])
Accuracy-(tensor(984, device='cuda:7'), 1176)
tensor(96.0156, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[379.,  18.,  15.],
        [ 11., 325.,  52.],
        [ 11.,  90., 275.]])
Accuracy-(tensor(979, device='cuda:7'), 1176)
tensor(95.1720, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[387.,  16.,  10.],
        [ 13., 313.,  62.],
        [ 16.,  97., 262.]])
Accuracy-(tensor(962, device='cuda:7'), 1176)
tensor(95.7999, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  19.,   9.],
        [ 10., 328.,  50.],
        [ 12.,  99., 264.]])
Accuracy-(tensor(977, device='cuda:7'), 1176)
tensor(96.6721, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[396.,  10.,   7.],
        [  9., 329.,  49.],
        [ 19.,  78., 279.]])
Accuracy-(tensor(1004, device='cuda:7'), 1176)
tensor(104.5784, device='cuda:7', grad_fn=<AddBackward0>)
t

tensor(102.1418, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[393.,  13.,   7.],
        [ 10., 315.,  63.],
        [ 19.,  80., 276.]])
Accuracy-(tensor(984, device='cuda:7'), 1176)
tensor(95.6715, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[393.,   9.,  11.],
        [ 11., 309.,  67.],
        [ 15.,  84., 277.]])
Accuracy-(tensor(979, device='cuda:7'), 1176)
tensor(96.9118, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  19.,   9.],
        [  5., 349.,  34.],
        [ 15., 102., 258.]])
Accuracy-(tensor(992, device='cuda:7'), 1176)
tensor(99.2732, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[393.,   8.,  12.],
        [ 10., 333.,  45.],
        [ 20.,  79., 276.]])
Accuracy-(tensor(1002, device='cuda:7'), 1176)
tensor(91.1824, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[387.,  15.,  10.],
        [  8., 322.,  58.],
        [ 14.,  80., 282.]])
Accuracy-(tensor(991, device='cuda:7'), 1176)
tensor(99.0230, device='cuda:7', grad_fn=<AddBackward0>)
t

tensor(97.8460, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[393.,  12.,   7.],
        [ 13., 327.,  48.],
        [ 19.,  64., 293.]])
Accuracy-(tensor(1013, device='cuda:7'), 1176)
tensor(93.7306, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  11.,  16.],
        [ 13., 312.,  63.],
        [ 24.,  69., 283.]])
Accuracy-(tensor(980, device='cuda:7'), 1176)
tensor(96.8960, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[383.,  16.,  14.],
        [ 12., 328.,  47.],
        [ 17.,  81., 278.]])
Accuracy-(tensor(989, device='cuda:7'), 1176)
tensor(96.3949, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[383.,  18.,  12.],
        [  9., 337.,  41.],
        [ 18.,  90., 268.]])
Accuracy-(tensor(988, device='cuda:7'), 1176)
tensor(97.3571, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[397.,   9.,   6.],
        [  5., 341.,  42.],
        [ 14.,  81., 281.]])
Accuracy-(tensor(1019, device='cuda:7'), 1176)
tensor(103.0854, device='cuda:7', grad_fn=<AddBackward0>)


tensor(100.8391, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[391.,  12.,   9.],
        [ 11., 313.,  64.],
        [ 15.,  83., 278.]])
Accuracy-(tensor(982, device='cuda:7'), 1176)
tensor(92.8708, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[398.,   7.,   7.],
        [ 13., 326.,  49.],
        [ 14., 103., 259.]])
Accuracy-(tensor(983, device='cuda:7'), 1176)
tensor(98.2542, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,  16.,  12.],
        [ 11., 329.,  47.],
        [ 19.,  93., 264.]])
Accuracy-(tensor(978, device='cuda:7'), 1176)
tensor(91.6188, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[396.,  11.,   5.],
        [ 10., 334.,  44.],
        [ 19.,  84., 273.]])
Accuracy-(tensor(1003, device='cuda:7'), 1176)
tensor(99.6894, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[403.,   5.,   5.],
        [ 12., 316.,  60.],
        [ 24.,  69., 282.]])
Accuracy-(tensor(1001, device='cuda:7'), 1176)
tensor(97.0660, device='cuda:7', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
!nvidia-smi
tensor([[396.,   9.,   7.],
        [  9., 332.,  47.],
        [ 16.,  77., 283.]])
Accuracy-(tensor(1011, device='cuda:7'), 1176)